In [3]:
import pickle
import numpy as np
import os
import csv
import matplotlib.pyplot as plt

# ================= 配置区域 =================
DATA_DIR = './data'
TRAIN_PATH = os.path.join(DATA_DIR, 'train_data.pkl')
TEST_PATH = os.path.join(DATA_DIR, 'test_data.pkl')

# 训练超参数
HIDDEN_SIZE = 512
NUM_CLASSES = 5
LEARNING_RATE = 0.05
MOMENTUM = 0.9
REG = 0.001
BATCH_SIZE = 64
EPOCHS = 40
NUM_MODELS = 5

np.random.seed(42)
print("环境配置完成。")

# ================= 数据处理工具 =================
def load_and_process_data(path, has_labels=True, target_max_size=64):
    if not os.path.exists(path):
        raise FileNotFoundError(f"{path} 未找到")
    
    # 修正点：确保这里没有多余的符号
    with open(path, 'rb') as f:
        data = pickle.load(f)
    
    images = np.array(data['images'])
    
    # 确保格式为 (N, C, H, W)
    if images.shape[-1] == 3:
        images = images.transpose(0, 3, 1, 2)
    
    N, C, H, W = images.shape
    print(f"加载文件: {path}, 原始形状: {images.shape}")
    
    # 动态计算步长，防止除以0错误
    h_step = max(1, H // target_max_size)
    w_step = max(1, W // target_max_size)
    
    # 执行切片降采样
    images_processed = images[:, :, ::h_step, ::w_step]
    
    # 归一化 [0, 1]
    X = images_processed.astype(float) / 255.0
    
    # 展平
    X_flat = X.reshape(N, -1)
    
    y = np.array(data['labels']) if has_labels else None
    
    return X_flat, y, images_processed.shape[1:]

def augment_batch(X_batch, img_shape):
    """随机水平翻转 (Data Augmentation)"""
    N = X_batch.shape[0]
    C, H, W = img_shape
    
    mask = np.random.rand(N) > 0.5
    if not np.any(mask):
        return X_batch
    
    X_img = X_batch.reshape(N, C, H, W)
    # 翻转宽度维度
    X_img[mask] = X_img[mask, :, :, ::-1]
    
    return X_img.reshape(N, -1)

# ================= 1. 加载数据 =================
print("正在加载数据...")
# [cite_start]加载训练集 [cite: 14]
X_train_all, y_train_all, img_shape = load_and_process_data(TRAIN_PATH, has_labels=True)
# [cite_start]加载测试集 [cite: 16]
X_test_raw, _, _ = load_and_process_data(TEST_PATH, has_labels=False)

INPUT_SIZE = X_train_all.shape[1]
print(f"最终输入特征维度 (INPUT_SIZE): {INPUT_SIZE}")

# 全局标准化
mean_vals = np.mean(X_train_all, axis=0)
std_vals = np.std(X_train_all, axis=0) + 1e-7

X_train_all = (X_train_all - mean_vals) / std_vals
X_test_norm = (X_test_raw - mean_vals) / std_vals
print("数据标准化完成。")

# ================= 2. 定义模型 (NumPy Only) =================
class TwoLayerNet:
    def __init__(self, input_dim, hidden_dim, output_dim, std=1e-2):
        self.params = {}
        # He Initialization
        self.params['W1'] = np.random.randn(input_dim, hidden_dim) * np.sqrt(2.0/input_dim)
        self.params['b1'] = np.zeros(hidden_dim)
        self.params['W2'] = np.random.randn(hidden_dim, output_dim) * np.sqrt(2.0/hidden_dim)
        self.params['b2'] = np.zeros(output_dim)
        
        self.v = {k: np.zeros_like(v) for k, v in self.params.items()}

    def loss(self, X, y=None, reg=0.0):
        W1, b1 = self.params['W1'], self.params['b1']
        W2, b2 = self.params['W2'], self.params['b2']
        N = X.shape[0]

        # Forward
        h1 = np.maximum(0, X.dot(W1) + b1)
        scores = h1.dot(W2) + b2
        
        if y is None:
            return scores

        # Softmax Loss
        shifted_scores = scores - np.max(scores, axis=1, keepdims=True)
        exp_scores = np.exp(shifted_scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        
        correct_logprobs = -np.log(probs[np.arange(N), y] + 1e-9)
        loss = np.sum(correct_logprobs) / N
        loss += 0.5 * reg * (np.sum(W1*W1) + np.sum(W2*W2))

        # Backward
        grads = {}
        dscores = probs
        dscores[np.arange(N), y] -= 1
        dscores /= N

        grads['W2'] = h1.T.dot(dscores) + reg * W2
        grads['b2'] = np.sum(dscores, axis=0)

        dh1 = dscores.dot(W2.T)
        dh1[h1 <= 0] = 0

        grads['W1'] = X.T.dot(dh1) + reg * W1
        grads['b1'] = np.sum(dh1, axis=0)

        return loss, grads

    def update(self, grads, learning_rate, momentum=0.9):
        for p in self.params:
            self.v[p] = momentum * self.v[p] - learning_rate * grads[p]
            self.params[p] += self.v[p]

    def predict_scores(self, X):
        return self.loss(X)

# ================= 3. 集成训练 (5 Models) =================
models = []
print(f"\n开始训练集成模型 (共 {NUM_MODELS} 个)...")

indices = np.arange(len(X_train_all))
fold_size = len(X_train_all) // NUM_MODELS

for i in range(NUM_MODELS):
    # 划分 Fold
    val_idx = indices[i*fold_size : (i+1)*fold_size]
    train_idx = np.concatenate([indices[:i*fold_size], indices[(i+1)*fold_size:]])
    
    X_tr_fold = X_train_all[train_idx]
    y_tr_fold = y_train_all[train_idx]
    X_val_fold = X_train_all[val_idx]
    y_val_fold = y_train_all[val_idx]
    
    print(f"--- Training Model {i+1} ---")
    net = TwoLayerNet(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES)
    
    iter_per_epoch = max(len(X_tr_fold) // BATCH_SIZE, 1)
    num_iters = EPOCHS * iter_per_epoch
    current_lr = LEARNING_RATE
    
    for it in range(num_iters):
        batch_mask = np.random.choice(len(X_tr_fold), BATCH_SIZE)
        X_batch = X_tr_fold[batch_mask]
        y_batch = y_tr_fold[batch_mask]
        
        # Augment
        X_batch_aug = augment_batch(X_batch, img_shape)
        
        loss, grads = net.loss(X_batch_aug, y_batch, reg=REG)
        net.update(grads, current_lr, momentum=MOMENTUM)
        
        if it > 0 and it % (10 * iter_per_epoch) == 0:
            current_lr *= 0.8
            
    val_preds = np.argmax(net.predict_scores(X_val_fold), axis=1)
    acc = np.mean(val_preds == y_val_fold)
    print(f"Model {i+1} Val Acc: {acc:.4f}")
    
    models.append(net)

# ================= 4. 预测与提交 =================
print("\n正在生成最终预测...")
total_scores = np.zeros((X_test_norm.shape[0], NUM_CLASSES))

for net in models:
    total_scores += net.predict_scores(X_test_norm)

final_predictions = np.argmax(total_scores, axis=1)

filename = 'submission_ensemble_fixed.csv'
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID', 'Label'])
    for i, label in enumerate(final_predictions):
        writer.writerow([i + 1, label])

print(f"成功生成: {filename}")

环境配置完成。
正在加载数据...
加载文件: ./data\train_data.pkl, 原始形状: (1080, 3, 28, 28)
加载文件: ./data\test_data.pkl, 原始形状: (400, 3, 28, 28)
最终输入特征维度 (INPUT_SIZE): 2352
数据标准化完成。

开始训练集成模型 (共 5 个)...
--- Training Model 1 ---


C:\Users\yudim\AppData\Local\Temp\ipykernel_22068\2484305379.py:112: RuntimeWarning: overflow encountered in dot
  scores = h1.dot(W2) + b2
C:\Users\yudim\AppData\Local\Temp\ipykernel_22068\2484305379.py:118: RuntimeWarning: invalid value encountered in subtract
  shifted_scores = scores - np.max(scores, axis=1, keepdims=True)


Model 1 Val Acc: 0.4259
--- Training Model 2 ---
Model 2 Val Acc: 0.4630
--- Training Model 3 ---
Model 3 Val Acc: 0.4861
--- Training Model 4 ---
Model 4 Val Acc: 0.4120
--- Training Model 5 ---
Model 5 Val Acc: 0.4630

正在生成最终预测...
成功生成: submission_ensemble_fixed.csv
